# Import libraries

In [7]:
#---------------------------------------------------------------------------------------------------------------
#IMPORT LIBRARIES


import os
import inro.emme.desktop.app as _app #import inro.emme.desktop as _app
import inro.modeller as m


# Initialize Modeller & emmebank objects

In [8]:
#Initialize modeller and emme.database
mm = m.Modeller()
from inro.emme.matrix import MatrixData
eb = mm.emmebank


# Load trip matrices from output folder

In [18]:
#launch matrix butler to export selected trip matrices to the emmebank
#matrix_list provides the list of matrices to be exported
#[[number,"mf id", 'name of matrix']]




seed_mat_list=[[1,'mf01',"peak_auto_demand_tts_plus_externals.mdf",'peak_auto_demand_tts_plus_externals'],
                [2,'mf02',"offpeak_auto_demand_tts_plus_externals.mdf",'offpeak_auto_demand_tts_plus_externals'],
                [3,'mf03',"peak_train_demand_tts.mdf",'peak_train_demand_tts'],
                [4,'mf04',"peak_subway_demand_tts.mdf",'peak_subway_demand_tts'],
                [5,'mf05',"peak_ztsa_demand_tts.mdf",'peak_ztsa_demand_tts'],
                [6,'mf06',"peak_express_bus_demand_tts.mdf",'2011_peak_express_bus_demand_tts'],
                [7,'mf07',"peak_rapid_bus_demand_tts.mdf",'peak_rapid_bus_demand_tts'],
                [8,'mf08',"peak_bus_streetcar_demand_tts.mdf",'peak_bus_streetcar_demand_tts']]
                

# UPDATE THE INPUT AND OUTPUT PATHS 
# input_path is the folder to read matrices from
# save_path is the folder to save matrices to
input_path = "F:\\GGHM v4 Databanks\\Prototype GGHM4\\Inputs\\Base - 2011\\Seed Matrices\\"
save_path = "F:\\GGHM v4 Databanks\\Prototype GGHM4\\Inputs\\Base - 2041 test\\Seed Matrices\\"
scen_in = 101
scen_out = 4100


#create full matrices to store seed matrices from 2011
for mat in seed_mat_list:
    
    matrix_filename = os.path.join(input_path,mat[2]) #save matrix file name into a variable
    md = MatrixData.load(matrix_filename)                 #load matrix from file
    
    if eb.matrix(mat[1]) is None: #Check if the matrix exists; if not, create it
        NAMESPACE = "inro.emme.data.matrix.create_matrix"
        create_matrix = mm.tool(NAMESPACE)
        new_mat = create_matrix(matrix_id=mat[1],
                        matrix_name=mat[3],
                        matrix_description=mat[3],
                        default_value=0)
        matrix = eb.matrix(mat[1])
        matrix.name = mat[3] #update matrix name
        matrix.description = mat[3] #update matrix description
        matrix.set_data(md,scenario_id=scen_in) #set matrix values into mf'x'
    else:
        matrix = eb.matrix(mat[1])
        matrix.name = mat[3] #update matrix name
        matrix.description = mat[3] #update matrix description
        matrix.set_data(md,scenario_id = scen_in) #set matrix values into mf'x'
        
#load auto & transit matrices and update matrices names
for mat in seed_mat_list:
   
    Export_Filename = os.path.join(save_path,mat[2])
    matrix_data = eb.matrix(mat[1]).get_data(scen_out)
    matrix_data.save(Export_Filename)
        

# Export trip matrices to *.csv files for SPD partition

In [17]:
#EXPORT DEMAND MATRICES INTO EXCEL FILES 
    # demand is aggregated to the 'gg' ensemble 
    # square table formats
    
#---------------------------------------------------------------------------------------------------------------
# Export auto demand matrices in aggregated to a 'gg' ensemble - AM, midday, PM, evening & overnight
NAMESPACE = "inro.emme.data.matrix.export_matrices"
export_matrices = m.Modeller().tool(NAMESPACE)

base_scenario = mm.emmebank.scenario(scen_out)
matrices_file = os.path.join(save_path,"2041_seed_auto_matrices.csv")
export_matrices(matrices=["mf01","mf02","mf03","mf04","mf05","mf06","mf07","mf08"],
                export_file=matrices_file,
                partition_aggregation = {'operator': 'sum', 'origins': u'gs', 'destinations': u'gs'},
                field_separator=',',
                export_format="PROMPT_DATA_FORMAT",
                skip_default_values=True,
                full_matrix_line_format="SQUARE",
                scenario = base_scenario)



print "GGHM Seed matrices exported to: " + matrices_file

TypeError: cannot concatenate 'str' and 'module' objects